In [1]:
pip install spotipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 4.8 MB/s eta 0:00:00


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Obtener credenciales con este tutorial https://youtu.be/j4J7B1C3G0U
client_id = '0ade2b525de24507b6ffac8ce3724b54' # Cambiar por tu Client ID
client_secret = 'f50981db7c9843949c30b5930acf9898' # Cambiar por tu Client Secret

# Autenticación
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
def encontrarID(artista):
    resultados = sp.search(q=artista, limit=1, type='artist')
    return resultados['artists']['items'][0]['id']

def obtenerData(artista_id): # 'album', 'single', 'appears_on', 'compilation'
    canciones_data = [] # Crear una lista vacía para almacenar los datos de las canciones

    albumes = sp.artist_albums(artista_id, album_type='album') # Obtener los álbumes del artista
    singles = sp.artist_albums(artista_id, album_type='single') # Obtener los singles del artista

    for album in albumes['items'] + singles['items']: # Obtener las canciones para cada álbum
        album_nombre = album['name']
        album_tipo = {'album':'Álbum', 'single':'Single'}[album['album_type']]
        album_año = album['release_date'].split('-')[0]  # Obtener el año de lanzamiento

        tracks = sp.album_tracks(album['id'])

        for track in tracks['items']: # Para cada canción, obtener los detalles y añadirlos a la lista de canciones
            cancion_nombre = track['name']
            cancion_artistas = ', '.join([t['name'] for t in track['artists']])
            cancion_duracion = '{:02d}:{:02d}'.format(*divmod(track['duration_ms'] // 1000, 60))
            cancion_popularidad = sp.track(track['id'])['popularity'] # Obtener la popularidad de la canción

            # Añadir los detalles de cadaa canción a la lista de canciones
            canciones_data.append([album_nombre, album_tipo, album_año, cancion_nombre, cancion_artistas, cancion_duracion, cancion_popularidad])

    return pd.DataFrame(canciones_data, columns=['Álbum', 'Tipo', 'Año', 'Canción', 'Artistas', 'Duración', 'Popularidad'])

In [9]:
tu_artista = 'Bad Omens'
id_artista = encontrarID(tu_artista)
print(f'ID de {tu_artista}: {id_artista}')

df = obtenerData(id_artista)
df

ID de Bad Omens: 3Ri4H12KFyu98LMjSoij5V


,Álbum,Tipo,Año,Canción,Artistas,Duración,Popularidad
0,THE DEATH OF PEACE OF MIND,Álbum,2022,CONCRETE JUNGLE,Bad Omens,03:40,67
1,THE DEATH OF PEACE OF MIND,Álbum,2022,Nowhere To Go,Bad Omens,04:06,69
2,THE DEATH OF PEACE OF MIND,Álbum,2022,Take Me First,Bad Omens,03:19,65
3,THE DEATH OF PEACE OF MIND,Álbum,2022,THE DEATH OF PEACE OF MIND,Bad Omens,04:01,76
4,THE DEATH OF PEACE OF MIND,Álbum,2022,What It Cost,Bad Omens,01:43,63
...,...,...,...,...,...,...,...
72,Suffocate,Single,2020,Suffocate,"Kayzo, Bad Omens",02:59,60
73,Never Know,Single,2019,Never Know,Bad Omens,03:33,30
74,The Fountain,Single,2016,The Fountain,Bad Omens,03:59,28
75,Exit Wounds,Single,2016,Exit Wounds,Bad Omens,03:26,30


In [10]:
df_canciones = df.copy()
df_canciones['Duración_seg'] = df_canciones['Duración'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))
df_canciones.head()

,Álbum,Tipo,Año,Canción,Artistas,Duración,Popularidad,Duración_seg
0,THE DEATH OF PEACE OF MIND,Álbum,2022,CONCRETE JUNGLE,Bad Omens,03:40,67,220
1,THE DEATH OF PEACE OF MIND,Álbum,2022,Nowhere To Go,Bad Omens,04:06,69,246
2,THE DEATH OF PEACE OF MIND,Álbum,2022,Take Me First,Bad Omens,03:19,65,199
3,THE DEATH OF PEACE OF MIND,Álbum,2022,THE DEATH OF PEACE OF MIND,Bad Omens,04:01,76,241
4,THE DEATH OF PEACE OF MIND,Álbum,2022,What It Cost,Bad Omens,01:43,63,103


In [11]:
canciones_populares = df_canciones.sort_values(by=['Popularidad'], ascending=False).head(15)
canciones_populares['Canción_única'] = canciones_populares['Canción'] + '<br>' + canciones_populares['Álbum']

color_dict = {'Álbum': '#1ED660', 'Single': 'white'}

# Creamos el gráfico de barras
fig = px.bar(canciones_populares,
             x='Canción_única',
             y='Popularidad',
             text='Popularidad',
             color='Tipo',
             color_discrete_map=color_dict,
             hover_data={'Álbum':True, 'Año':True, 'Artistas':True},
             title='Canciones más populares',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Canciones más populares</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis={'categoryorder': 'total descending'},
                  yaxis={'categoryorder': 'total descending'})

# Establecemos el rango del eje Y
fig.update_yaxes(range=[70, canciones_populares['Popularidad'].max()+3], showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(textposition='outside')

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.17, sizey=0.17,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()


In [14]:
df_canciones.groupby(['Tipo']).agg({'Canción':'count'}).reset_index()

,Tipo,Canción
0,Single,20
1,Álbum,57


In [15]:
canciones_tipo = df_canciones.groupby(['Tipo']).agg({'Canción':'count'}).reset_index()

color_dict = {'Álbum': '#1ED660', 'Single': 'white'}

# Creamos el gráfico de barras
fig = px.bar(canciones_tipo,
             x='Tipo',
             y='Canción',
             text='Canción',
             color='Tipo',
             color_discrete_map=color_dict,
             labels={'Canción':'Canciones'},
             title='Popularidad Promedio de las Canciones por Álbum',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes vs. Singles</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis={'categoryorder': 'total descending'},
                  yaxis={'categoryorder': 'total descending'})

# Establecemos el rango del eje Y
fig.update_yaxes(showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(textposition='inside', insidetextanchor='middle', textfont=dict(size=18),
                  width=0.4)

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.15, sizey=0.15,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

In [16]:
df_albumes = df.copy()
df_albumes['Duración_seg'] = df_albumes['Duración'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))
df_albumes = df_albumes.groupby('Álbum').agg({'Tipo':'min', 'Año':'min', 'Popularidad':'mean', 'Duración_seg':'sum', 'Canción':'count'})
df_albumes['Duración'] = df_albumes['Duración_seg'].apply(lambda x:'{:02d}:{:02d}'.format(*divmod(int(x), 60)))
df_albumes.head()

,Tipo,Año,Popularidad,Duración_seg,Canción,Duración
Álbum,,,,,,
Bad Omens,Álbum,2016,52.416667,2599,12,43:19
Exit Wounds,Single,2016,30.000000,206,1,03:26
FGBGFM Unplugged,Single,2020,40.666667,1480,6,24:40
Finding God Before God Finds Me,Álbum,2019,53.500000,2529,10,42:09
Finding God Before God Finds Me (Deluxe),Álbum,2020,51.153846,3210,13,53:30


In [17]:
albumes_populares = df_albumes.sort_values(by=['Popularidad'], ascending=False).reset_index().head(20)

color_dict = {'Álbum': '#1ED660', 'Single': 'white'}

# Creamos el gráfico de barras
fig = px.bar(albumes_populares,
             x='Álbum',
             y='Popularidad',
             text='Popularidad',
             color='Tipo',
             color_discrete_map=color_dict,
             hover_data={'Año':True},
             title='Álbumes más populares',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes más populares</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis={'categoryorder': 'total descending'})

# Establecemos el rango del eje Y
fig.update_yaxes(range=[40, albumes_populares['Popularidad'].max()+4], showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_Spotify.png",
                          xref="paper", yref="paper",x=0.96, y=1.14,
                          sizex=0.22, sizey=0.22,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

In [18]:
albumes_duracion = df_albumes.sort_values(by=['Duración_seg'], ascending=False).reset_index().head(15)
albumes_duracion['Duración_prom_seg'] = albumes_duracion['Duración_seg'] / albumes_duracion['Canción']
albumes_duracion['Duración_prom'] = albumes_duracion['Duración_prom_seg'].apply(lambda x:'{:02d}:{:02d}'.format(*divmod(int(x), 60)))
albumes_duracion

,Álbum,Tipo,Año,Popularidad,Duración_seg,Canción,Duración,Duración_prom_seg,Duración_prom
0,Finding God Before God Finds Me (Deluxe),Álbum,2020,51.153846,3210,13,53:30,246.923077,04:06
1,THE DEATH OF PEACE OF MIND,Álbum,2022,65.133333,3180,15,53:00,212.000000,03:32
2,Bad Omens,Álbum,2016,52.416667,2599,12,43:19,216.583333,03:36
3,Finding God Before God Finds Me,Álbum,2019,53.500000,2529,10,42:09,252.900000,04:12
4,LIVE,Álbum,2021,36.857143,1893,7,31:33,270.428571,04:30
5,FGBGFM Unplugged,Single,2020,40.666667,1480,6,24:40,246.666667,04:06
6,Novocaine,Single,2024,56.000000,772,3,12:52,257.333333,04:17
7,Never Know (Live),Single,2021,27.500000,424,2,07:04,212.000000,03:32
8,Limits,Single,2020,37.500000,403,2,06:43,201.500000,03:21
9,V.A.N,Single,2024,72.000000,274,1,04:34,274.000000,04:34


In [19]:
albumes_duracion = df_albumes.sort_values(by=['Duración_seg'], ascending=False).reset_index().head(15)
albumes_duracion['Duración_prom_seg'] = albumes_duracion['Duración_seg'] / albumes_duracion['Canción']
albumes_duracion['Duración_prom'] = albumes_duracion['Duración_prom_seg'].apply(lambda x:'{:02d}:{:02d}'.format(*divmod(int(x), 60)))

color_dict = {'Álbum': '#1ED660', 'Single': 'white'}

# Creamos el gráfico de barras
fig = px.bar(albumes_duracion,
             x='Álbum',
             y='Duración_seg',
             text='Duración',
             color='Tipo',
             color_discrete_map=color_dict,
             labels={'Duración_seg':'Duración'},
             hover_data={'Año':True, 'Tipo':True},
             title='Álbumes con más duración',
             template='plotly_dark')

# Ajustar posición y formato del texto
fig.update_traces(textposition='outside')

# Añadir un gráfico de líneas con la duración promedio en el eje y secundario
fig.add_trace(go.Scatter(x=albumes_duracion['Álbum'],
                         y=albumes_duracion['Duración_prom_seg'],
                         mode='lines+markers',
                         yaxis='y2',
                         name='Duración Promedio',
                         line=dict(color='#F3E5AB'),
                         text='Duración Promedio= ' + albumes_duracion['Duración_prom'],
                         hoverinfo='text'))

# Establecemos el rango del eje Y
fig.update_yaxes(range=[0, albumes_duracion['Duración_seg'].max()+400])



# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes con más duración</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  yaxis={'categoryorder': 'total descending'},
                  yaxis2=dict(title='Duración promedio',
                              overlaying='y',
                              side='right',
                              range=[150, albumes_duracion['Duración_prom_seg'].max()+150]),
                  margin=dict(b=270)) # Usar margin cuando los valores del eje x son muy largos

# Establecemos el rango del eje Y
fig.update_yaxes(showgrid=False, showticklabels=False, title='')

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.21, sizey=0.21,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

In [20]:
albumes_canciones = df_albumes[df_albumes['Tipo']=='Álbum'].sort_values(by=['Canción'], ascending=False).reset_index()

color_dict = {'Álbum': '#1ED660', 'Single': 'white'}

# Creamos el gráfico de barras
fig = px.bar(albumes_canciones,
             x='Álbum',
             y='Canción',
             text='Canción',
             color='Tipo',
             color_discrete_map=color_dict,
             labels={'Canción':'# Canciones'},
             hover_data={'Año':True},
             title='Álbumes con más canciones',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes con más canciones</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis={'categoryorder': 'total descending'},
                  yaxis={'categoryorder': 'total descending'})

# Establecemos el rango del eje Y
fig.update_yaxes(range=[5, albumes_canciones['Canción'].max()+2], showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(textposition='outside')

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.2, sizey=0.2,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()


In [21]:
albumes_singles = df_albumes.reset_index().groupby(['Tipo']).agg({'Álbum':'count'}).reset_index()

color_dict = {'Álbum': '#1ED660', 'Single': 'white'}

# Creamos el gráfico de barras
fig = px.bar(albumes_singles,
             x='Tipo',
             y='Álbum',
             text='Álbum',
             color='Tipo',
             color_discrete_map=color_dict,
             labels={'Álbum':'Cantidad'},
             title='Álbumes vs. Singles',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes vs. Singles</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis={'categoryorder': 'total descending'},
                  yaxis={'categoryorder': 'total descending'})

# Establecemos el rango del eje Y
fig.update_yaxes(showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(textposition='inside', insidetextanchor='middle', textfont=dict(size=18),
                  width=0.4)

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.16, sizey=0.16,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

In [22]:
df_albumes.sort_values(by=['Año','Tipo'], ascending=False).reset_index()[['Álbum','Tipo','Año']]


,Álbum,Tipo,Año
0,Novocaine,Single,2024
1,V.A.N,Single,2024
2,THE DEATH OF PEACE OF MIND,Álbum,2022
3,LIVE,Álbum,2021
4,Never Know (Live),Single,2021
5,Finding God Before God Finds Me (Deluxe),Álbum,2020
6,FGBGFM Unplugged,Single,2020
7,Limits,Single,2020
8,Never Know (Unplugged),Single,2020
9,Suffocate,Single,2020
